# Working with Supermarket Big Data

This notebook uses a subset of our large dataset of British supermarket price micro data. The data possesses the some of teh characteristics of Big Data.

- **Volume**: Millions of rows from a dataset of tens of millions.
- **Velocity**: Data generated and processed in real time, updated daily.
- **Variety**: Diverse formats, including numeric and categorical data, with anonymised product and store identifiers.



In [1]:
# Import Required Libraries
import pandas as pd
import numpy as np
import altair as alt

## Loading Data

Typically, you would load Big Data from a database or alternate source. Today, we will be parquette files instead - a modern and efficent data format.

</br>
</br>

Two sources are provided:

- **Prices** Daily price observations for British supermarket products. The prices are identified according to via an ID for the store and product.
- **Items** Descriptive classification information for the products. The products are identified by their equivalent 'segment' in the CPI bsaket but their exact product name and store identity are anonymised.

In [54]:
# Load cleaned prices and items data
prices_df = pd.read_parquet('https://autocpi-public.s3.eu-west-2.amazonaws.com/pp434/pp434_semi_anonymised_prices.parquet')
items_df = pd.read_parquet('https://autocpi-public.s3.eu-west-2.amazonaws.com/pp434/pp434_semi_anonymised_items.parquet')

## Preview Data Samples

Let's inspect a few rows from each dataset to understand their structure.

In [21]:
# Show sample rows from prices_df and items_df
prices_df.sample(5)

,store_id,product_id,date,price,unit_price,loyalty_price,original_price
3531023,5,4088600124773,2023-09-06,1.39,£0.37 / 100g,NaN,NaN
29299266,8,7041219996099961,2024-09-01,5.00,£15.63/100g,NaN,NaN
18040536,9,90380,2024-10-06,2.50,74p per 100 gram,NaN,NaN
12680686,3,1000383145776,2024-07-14,1.50,33.3p/100g,NaN,1.95
1014421,4,102368906,2024-11-23,1.50,nan,NaN,NaN


In [22]:
items_df.sample(5)

,store_id,product_id,segment_code,description
102558,4,578078011,CP0211001,"SPIRITS, GIN"
92552,3,1000383166436,CP0118902,"SWEETS, SOFT AND HARD (EXCL. MINTS AND CHEWING..."
65910,1,253174043,CP0119304,PRE-MADE SAUCES (E.G. BOLOGNESE)
92584,1,314848147,CP0117903,"CRISPS, POTATO"
41906,1,311447330,CP0213003,"BEER, ALE"


# Associating the dataframes

Our `prices_df` contains prices and ids for the store (`store_id`) and product (`product_id`) but it would be easier to work from a dataframe that includes product informaton as well, which is contained in `items_df`.

</br></br>

Let's associate the data with a merge.

In [23]:
# Merge prices and items on store_id and product_id
df = pd.merge(prices_df, items_df, on=['store_id', 'product_id'], how='inner')
df.sample(5)

,store_id,product_id,date,price,unit_price,loyalty_price,original_price,segment_code,description
4194863,4,114743857,2025-11-11,2.50,0.86/100gram,NaN,3.0,CP0111311,"CAKES, TARTS AND SWEET PIES"
23240844,2,3277881,2024-04-30,1.70,43p / 100g,NaN,1.7,CP0119106,READY-MADE SOUPS
24171309,3,1000287073968,2023-09-17,2.20,31.4p/100g,NaN,NaN,CP0119302,CONDIMENTS
7163973,3,910003166292,2024-12-30,3.00,£2.73/kg,NaN,NaN,CP0112204,"MEAT OF CHICKEN, FRESH, CHILLED OR FROZEN"
17717306,1,259247714,2024-06-13,3.65,0 per each,NaN,NaN,CP0118101,SUGAR


## Store-Level Prices

Let's examine how prices vary across stores by calculating median and mean prices for each store.

In [24]:
# Calculate median and mean prices for each store
store_prices = df.copy()
median_prices = store_prices.groupby(['store_id']).agg({'price': ['median', 'mean']}).reset_index()
median_prices.columns = ['store_id', 'median_price', 'mean_price']
median_prices

,store_id,median_price,mean_price
0,1,2.50,3.723194
1,2,2.50,3.819323
2,3,2.15,3.328313
3,4,2.15,3.266582
4,5,1.69,2.392112
5,8,2.50,3.633260
6,9,2.40,3.106512


## Charting store prices

Let's create a grouped bar chart to compare mean and median prices by store.

In [11]:
# Prepare data for grouped bar chart
median_prices_melted = median_prices.melt(id_vars='store_id', value_vars=['median_price', 'mean_price'], var_name='price_type', value_name='price')
median_prices_melted['store_id'] = "Store " + median_prices_melted['store_id'].astype(str)

# Plot grouped bar chart
alt.Chart(median_prices_melted).mark_bar().encode(
    column=alt.Column('store_id', title=''),
    x=alt.X('price_type', title='', axis=alt.Axis(labels=False)),
    y=alt.Y('price', title='', axis={"labelExpr": "'£' + datum.label", "labelOverlap": False}),
    color='price_type'
).properties(
    title = {
        'text': "Prices by store",
        'subtitle': ["Mean and median prices", ""]
    },
    width=100
)

alt.Chart(...)

</br></br>
</br></br>

### <b>Price distributions</b>

What does the price distribution of our dataset look like?

In [31]:
df.price.describe()

count    3.483554e+07
mean     3.467136e+00
std      4.368957e+00
min      2.000000e-01
25%      1.500000e+00
50%      2.300000e+00
75%      3.500000e+00
max      6.400000e+01
Name: price, dtype: float64

Can we display this more intuitively? Let's make a histogram.

Let's show prices in 10p bins from £0-10

In [32]:
# Create a copy of the original DataFrame
hist_df = prices_df.copy()

# Round the 'price' column to 1 decimal place to group prices into rounded intervals
hist_df['rounded_price'] = hist_df['price'].round(1)

# Group by the rounded prices and count the occurrences of each rounded price
hist_df = hist_df.groupby('rounded_price').agg({'price': 'count'}).reset_index()

# Filter out rows where the rounded price is greater than 10
hist_df = hist_df.query("rounded_price <= 10")

# Rename the columns for clarity: 'rounded_price' to 'price', and the count to 'density'
hist_df.columns = ['price', 'density']

# Normalize the density values to calculate the relative frequency (density)
hist_df['density'] = hist_df['density'] / hist_df['density'].sum()

# Create a histogram using Altair
histogram = alt.Chart(hist_df).mark_bar(
    width=5
).encode(
    x=alt.X('price:Q',  title='', axis={"labelExpr": "'£'+datum.value"}),  # Bin the 'price' values into 20 bins
    y=alt.Y('density:Q', title='Density'),  # Plot the normalized density on the y-axis,
    tooltip=['price', 'density']  # Show the 'price' and 'density' values on hover
)

# Display the histogram
histogram

alt.Chart(...)

</br> </br>

This is interesting. Can we Copy-Paste this code to loop over all our stores?

In [33]:
for store_id in prices_df.store_id.unique():
    temp_df = prices_df.query(f"store_id == {store_id}")
    # TODO: repeat the histogram code to create your own

## Price Distribution Analysis

Let's describe the price column and prepare data for histogram visualisation.

In [ ]:
# Describe price column
merged_df['price'].describe()

In [ ]:
# Prepare data for histogram: bin prices
hist_df = merged_df.copy()
hist_df['rounded_price'] = hist_df['price'].round(1)
hist_df = hist_df.groupby('rounded_price').agg({'price': 'count'}).reset_index()
hist_df = hist_df.query("rounded_price <= 10")
hist_df.columns = ['price', 'density']
hist_df['density'] = hist_df['density'] / hist_df['density'].sum()
hist_df.head()

## Visualize Price Distribution Histogram

Let's plot a histogram of price distribution using Altair.

In [ ]:
# Plot histogram of price distribution
histogram = alt.Chart(hist_df).mark_bar(width=5).encode(
    x=alt.X('price:Q',  title='', axis={"labelExpr": "'£'+datum.value"}),
    y=alt.Y('density:Q', title='Density'),
    tooltip=['price', 'density']
)
histogram

## Store-wise Price Distribution Loop

Let's loop over all stores and plot price distribution histograms for each.

In [ ]:
# Loop over stores and plot price histograms
for store_id in merged_df.store_id.unique():
    temp_df = merged_df.query(f"store_id == {store_id}")
    temp_hist = temp_df.copy()
    temp_hist['rounded_price'] = temp_hist['price'].round(1)
    temp_hist = temp_hist.groupby('rounded_price').agg({'price': 'count'}).reset_index()
    temp_hist = temp_hist.query("rounded_price <= 10")
    temp_hist.columns = ['price', 'density']
    temp_hist['density'] = temp_hist['density'] / temp_hist['density'].sum()
    chart = alt.Chart(temp_hist).mark_bar(width=5).encode(
        x=alt.X('price:Q',  title='', axis={"labelExpr": "'£'+datum.value"}),
        y=alt.Y('density:Q', title='Density'),
        tooltip=['price', 'density']
    ).properties(title=f"Store {store_id} Price Distribution")
    display(chart)

## An Example: Pizza

Let's filter for Pizza and see its prices across stores.

In [44]:
pizza_df = df[df['description'].str.contains('PIZZA', case=False, na=False)]

In [51]:
final_prices = pizza_df.drop_duplicates(subset=['store_id', 'product_id'], keep='last') # Keeping the last price for each store-product pair
final_prices['store_id'] = "Store " + final_prices['store_id'].astype(str) # Adding 'Store' to store_id for nicer labels
final_prices['jitter'] = np.random.rand(len(final_prices)) 

alt.Chart(final_prices).mark_circle(size=100).encode(
    y=alt.Y('store_id:N', title=''),
    x=alt.X('price:Q', title='Price (£)'),
    yOffset=alt.YOffset("jitter:Q"),
    color=alt.Color('store_id:N', legend=None),
).properties(
    width=500,
    height=400,
    title={
        'text': "Pizza prices by store",
        'subtitle': ["Most recent price for each product", ""],
        'anchor': 'start',
    }
)


/var/folders/p7/f_hqwzjd39zcyntdvmfs02zc0000gp/T/ipykernel_26384/1527649062.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_prices['store_id'] = "Store " + final_prices['store_id'].astype(str) # Adding 'Store' to store_id for nicer labels
/var/folders/p7/f_hqwzjd39zcyntdvmfs02zc0000gp/T/ipykernel_26384/1527649062.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_prices['jitter'] = np.random.rand(len(final_prices))


alt.Chart(...)